### Step 20: Integrate New COOP and ISD Stations

In [1]:
import arcpy,os;

fgdb = os.getcwd() + os.sep + 'source.gdb';

grid = fgdb + os.sep + 'CRWU_CREAT_Grid_Projections';
grid_cnt = arcpy.GetCount_management(grid)[0];
hist = fgdb + os.sep + 'CRWU_CREAT_Historic_Climate_Stations';
hist_cnt = arcpy.GetCount_management(hist)[0];
coop = fgdb + os.sep + 'COOP_STATIONS_TO_USE';
coop_cnt = arcpy.GetCount_management(coop)[0];
isd  = fgdb + os.sep + 'ISD_STATIONS_TO_USE';
isd_cnt = arcpy.GetCount_management(isd)[0];

print("Grid Projections : " + str(grid_cnt));
print("Historic Stations: " + str(hist_cnt));
print("COOP Stations    : " + str(coop_cnt));
print("ISD Stations     : " + str(isd_cnt));


Grid Projections : 24743
Historic Stations: 11165
COOP Stations    : 1851
ISD Stations     : 3293


In [2]:
wrkfgdb = os.getcwd() + os.sep + 'working.gdb';

if not arcpy.Exists(wrkfgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(wrkfgdb)
      ,os.path.basename(wrkfgdb)
   );

wrkstat = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK';

if arcpy.Exists(wrkstat):
    arcpy.Delete_management(wrkstat);
  
print("  creating new working stations");
arcpy.CreateFeatureclass_management(
     out_path      = wrkfgdb
    ,out_name      = 'D4EMLite_PREC_Details_WRK'
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table          = wrkstat
    ,field_description = [
         ['StationId'   ,'TEXT'  ,'StationId'   ,14 ]
        ,['DataType'    ,'TEXT'  ,'DataType'    ,255]
        ,['FileName'    ,'TEXT'  ,'FileName'    ,255]
        ,['Id'          ,'LONG'  ,'Id'              ]
        ,['Lat'         ,'DOUBLE','Lat'             ]
        ,['Long'        ,'DOUBLE','Long'            ]
        ,['Scenario'    ,'TEXT'  ,'Scenario'    ,255]
        ,['Constituent' ,'TEXT'  ,'Constituent' ,255]
        ,['SDate'       ,'TEXT'  ,'SDate'       ,12 ]
        ,['EDate'       ,'TEXT'  ,'EDate'       ,12 ]
        ,['YrCount'     ,'DOUBLE','YrCount'         ]
        ,['Value'       ,'DOUBLE','Value'           ]
        ,['StaNam'      ,'TEXT'  ,'StaNam'      ,255]
    ]
);

print("  adding indexes");
arcpy.management.AddIndex(
     in_table   = wrkstat
    ,fields     = 'StationId'
    ,index_name = 'StationId_IDX'
);

fldout = [
     'StationId'
    ,'DataType'
    ,'FileName'
    ,'Id'
    ,'Lat'
    ,'Long'
    ,'Scenario'
    ,'Constituent'
    ,'SDate'
    ,'EDate'
    ,'YrCount'
    ,'Value'
    ,'StaNam'
    ,'SHAPE@'
];

fldcoop = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'start_date_to_use_clean'
    ,'end_date_to_use_clean'
    ,'SHAPE@'
];

fldisd = [
     'station_id'
    ,'station_name'
    ,'state'
    ,'start_date_clean'
    ,'end_date_clean'
    ,'latitude'
    ,'longitude'
    ,'in_basins'
    ,'break_with_basins'
    ,'network'
    ,'year_count'
    ,'SHAPE@'
];

with arcpy.da.InsertCursor(wrkstat,fldout) as outcur:

    print("  loading COOP stations");
    with arcpy.da.SearchCursor(coop,fldcoop) as incur:
        for row in incur:
            
            outcur.insertRow((
                 row[0]
                ,'WdmFinal'
                ,row[2].lower() + str(row[0]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[11]
                ,row[12]
                ,row[10]
                ,0.0
                ,row[1]
                ,row[13]
            ));
            
    print("  loading ISD stations");
    with arcpy.da.SearchCursor(isd,fldisd) as incur:
        for row in incur:
            
            outcur.insertRow((
                 row[0]
                ,'WdmFinal'
                ,row[2].lower() + str(row[0]) + '.txt'
                ,1
                ,row[5]
                ,row[6]
                ,'OBSERVED'
                ,'PREC'
                ,row[3]
                ,row[4]
                ,row[10]
                ,0.0
                ,row[1]
                ,row[11]
            ));

print("DONE");


  creating new working stations
  adding indexes
  loading COOP stations
  loading ISD stations
DONE


In [3]:
stat_cnt = arcpy.GetCount_management(wrkstat)[0];

print("New Stations: " + str(stat_cnt));


New Stations: 5144
